In [44]:
import pandas as pd
import sqlite3 
import random

In [67]:
df = pd.read_csv("linkdin_Job_data.csv")

In [68]:
def replace_nulls(df):
    log = {"categorical": [], "numerical": [], "datetime": []}

    # Procesar columnas categóricas (object)
    for column in df.select_dtypes(include=['object']).columns:
        def fill_categorical_nulls(series):
            ffilled = series.ffill()
            bfilled = series.bfill()
            return series.fillna(ffilled.where(ffilled == bfilled, ffilled))
        
        df[column] = fill_categorical_nulls(df[column])
        log["categorical"].append({"column": column, "method": "ffill + bfill logic"})

    # Procesar columnas numéricas (int y float)
    for column in df.select_dtypes(include=['number']).columns:
        method = random.choice(["mean", "interpolation"])
        if method == "mean":
            df[column] = df[column].fillna(df[column].mean())
        else:
            df[column] = df[column].interpolate(method='linear', limit_direction='both')
        
        log["numerical"].append({"column": column, "method": method})

    # Procesar columnas datetime
    for column in [col for col in df.columns if "Date" in col or "Datetime" in col]:
        try:
            df[column] = pd.to_datetime(df[column], errors='coerce')
            df[column] = df[column].interpolate(method='time')
            log["datetime"].append({"column": column, "method": "interpolation", "converted_to_datetime": True})
        except Exception as e:
            log["datetime"].append({"column": column, "error": str(e)})

    return df


In [74]:
df = replace_nulls(df)
columns = ['company_id', 'Column1']
#df.drop(columns=columns, inplace=True)
df['Hiring_person'] = df['Hiring_person'].fillna('')

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7927 entries, 0 to 7926
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   job_ID              7927 non-null   int64 
 1   job                 7927 non-null   object
 2   location            7927 non-null   object
 3   company_name        7927 non-null   object
 4   work_type           7927 non-null   object
 5   full_time_remote    7927 non-null   object
 6   no_of_employ        7927 non-null   object
 7   no_of_application   7927 non-null   object
 8   posted_day_ago      7927 non-null   object
 9   alumni              7927 non-null   object
 10  Hiring_person       7927 non-null   object
 11  linkedin_followers  7927 non-null   object
 12  hiring_person_link  7925 non-null   object
 13  job_details         7927 non-null   object
dtypes: int64(1), object(13)
memory usage: 867.1+ KB


In [77]:
df = df.dropna()

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7925 entries, 2 to 7926
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   job_ID              7925 non-null   int64 
 1   job                 7925 non-null   object
 2   location            7925 non-null   object
 3   company_name        7925 non-null   object
 4   work_type           7925 non-null   object
 5   full_time_remote    7925 non-null   object
 6   no_of_employ        7925 non-null   object
 7   no_of_application   7925 non-null   object
 8   posted_day_ago      7925 non-null   object
 9   alumni              7925 non-null   object
 10  Hiring_person       7925 non-null   object
 11  linkedin_followers  7925 non-null   object
 12  hiring_person_link  7925 non-null   object
 13  job_details         7925 non-null   object
dtypes: int64(1), object(13)
memory usage: 928.7+ KB


In [79]:
connection = sqlite3.connect("linkedin.db")
df.to_sql("jobs", connection, if_exists="replace", index=False)
connection.close()

In [80]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [84]:
%sql sqlite:///jupyter.db

In [82]:
%%sql
SELECT * FROM jobs LIMIT 5

   sqlite:///jupyter.db
 * sqlite:///linkedin.db
Done.


KeyError: 'DEFAULT'

In [85]:
connection.close()